# Multiple Regression

This example will begin where the last chapter left off, except now we are adding new data into the model.  We will now propose a linear model with more independent variables:

$$ minutes = \alpha + \beta_1 friends + \beta_2 work hours + \beta_3 phd + \epsilon $$

Note that whether a user has a PhD is not a number; it is merely a *dummy variable* that equals 1 for users with PhDs and 0 for users without.

Recall the following model:

$$ y_1 = \alpha + \beta x_i + \epsilon_i $$

Imagine that each input $ x_i $ is not a single number but rather a vector of $ k $ numbers $ x_i1, ... , x_ik $.  The multiple regression model assumes:

$$ y_i = \alpha + \beta_1x_i1 + ... + \beta_k x_ik + \epsilon_i $$

In multiple regressions the vector of parameters is usually called $ \beta $.  We'll want this to include a constant term as well, which we can achieve by adding a column of ones to our data:

```python
beta = [alpha, beta_1, ..., beta_k]
x_i = [1, x_i1, ..., x_ik]
```

Then our model will be:

```python
def predict(x_i, beta):
    return dot(x_i, beta)
```

Which in this case our independent variable `x` will be a list of vectors, each of which look like this:

```python
[1,    # constant term
 49,   # number of friends
 4,    # work hours per day
 0]    # doesn't have a PhD
```

There are a couple of further assumptions that are required for this model to make sense:

1. Columns of `x` are *linearly independent*, meaning that there's no way to write any one as a weighted sum of some of the others.  If this assumption fails, it's impossible to estimate `beta`.  For example, imagine we had an extra field `num_acquaintances` in our data that for every user was exactly the same as `num_friends`.
2. Starting with any `beta`, if we add *any* amount to the `num_friends` coefficient and subtract that same amount from the `num_acquaintances` coefficient, the model's predictions will remain unchanged.  This means that there's no way to find the coefficient for `num_friends`.
3. The columns of `x` are all uncorrelated with the errors $ \epsilon $.  If this fails to be the case, our estimates of `beta` will be systematically wrong.  For example, we built a model that predicted that each additional friend was associated with an extra `0.90` daily minutes on the site.  Imagine that it's also the case that: people who work more hours spend less time on the site; and people with more friends tend to work more hours.  That is, imagine that the *actual* model is:

$$ minutes = \alpha + \beta_1friends + \beta_2work hours + \epsilon $$

and that work hours and friends are positively correlated.  In that case, when we minimize the errors of the single variable model

$$ minutes = \alpha + \beta_1friends + \epsilon $$

we will underestimate $ \beta_1 $.

So what does this mean?

If we made predictions using the single variable model with the "actual" value of `beta_1`, the predictions would tend to be too small for users who work many hours and too large for users who work few hours, because $ \beta_2 > 0 $ and we "forgot" to include it.  Because work hours is positively correlated with number of friends, this means the predictions tend to be too small for users with many friends and too large for users with few friends.

The result of this is that we can reduce the errors in the single variable model by decreasing our estimate of `beta_1`, which means that the error minimizing `beta_1` is smaller than the "actual" value.  So in this case, the single variable least squares solution is biased to underestimate `beta_1`.  And, in general, whenever the independent variables are correlated with the errors like this, our least squares solution will give us a biased estimate of $\beta$.

Just like in the simple linear model, we'll choose `beta` to minimize the sum of squared errors.  Finding an exact solution is not simple to do by hand, which means we'll need to use gradient descent.  We'll start by creating an error function to minimize.  For stochastic gradient descent, we'll just want the squared error corresponding to a single prediction:

In [8]:
from collections import Counter
from functools import partial
from code_python3.linear_algebra import dot, vector_add
from code_python3.stats import median, standard_deviation
from code_python3.probability import normal_cdf
from code_python3.gradient_descent import minimize_stochastic
from code_python3.simple_linear_regression import total_sum_of_squares
import math, random

def predict(x_i, beta):
    return dot(x_i, beta)

def error(x_i, y_i, beta):
    return y_i - predict(x_i, beta)

def squared_error(x_i, y_i, beta):
    return error(x_i, y_i, beta) ** 2

If you know calculus, you can compute:

In [6]:
def squared_error_gradient(x_i, y_i, beta):
    """the gradient corresponding to the ith squared error term"""
    return [-2 * x_ij * error(x_i, y_i, beta)
            for x_ij in x_i]

We're ready to find the optimal `beta` using stochastic gradient descent:

In [15]:
def estimate_beta(x, y):
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(squared_error,
                               squared_error_gradient,
                               x, y,
                               beta_initial,
                               0.001)

x = [[1,49,4,0],[1,41,9,0],[1,40,8,0],[1,25,6,0],[1,21,1,0],[1,21,0,0],[1,19,3,0],[1,19,0,0],[1,18,9,0],[1,18,8,0],[1,16,4,0],[1,15,3,0],[1,15,0,0],[1,15,2,0],[1,15,7,0],[1,14,0,0],[1,14,1,0],[1,13,1,0],[1,13,7,0],[1,13,4,0],[1,13,2,0],[1,12,5,0],[1,12,0,0],[1,11,9,0],[1,10,9,0],[1,10,1,0],[1,10,1,0],[1,10,7,0],[1,10,9,0],[1,10,1,0],[1,10,6,0],[1,10,6,0],[1,10,8,0],[1,10,10,0],[1,10,6,0],[1,10,0,0],[1,10,5,0],[1,10,3,0],[1,10,4,0],[1,9,9,0],[1,9,9,0],[1,9,0,0],[1,9,0,0],[1,9,6,0],[1,9,10,0],[1,9,8,0],[1,9,5,0],[1,9,2,0],[1,9,9,0],[1,9,10,0],[1,9,7,0],[1,9,2,0],[1,9,0,0],[1,9,4,0],[1,9,6,0],[1,9,4,0],[1,9,7,0],[1,8,3,0],[1,8,2,0],[1,8,4,0],[1,8,9,0],[1,8,2,0],[1,8,3,0],[1,8,5,0],[1,8,8,0],[1,8,0,0],[1,8,9,0],[1,8,10,0],[1,8,5,0],[1,8,5,0],[1,7,5,0],[1,7,5,0],[1,7,0,0],[1,7,2,0],[1,7,8,0],[1,7,10,0],[1,7,5,0],[1,7,3,0],[1,7,3,0],[1,7,6,0],[1,7,7,0],[1,7,7,0],[1,7,9,0],[1,7,3,0],[1,7,8,0],[1,6,4,0],[1,6,6,0],[1,6,4,0],[1,6,9,0],[1,6,0,0],[1,6,1,0],[1,6,4,0],[1,6,1,0],[1,6,0,0],[1,6,7,0],[1,6,0,0],[1,6,8,0],[1,6,4,0],[1,6,2,1],[1,6,1,1],[1,6,3,1],[1,6,6,1],[1,6,4,1],[1,6,4,1],[1,6,1,1],[1,6,3,1],[1,6,4,1],[1,5,1,1],[1,5,9,1],[1,5,4,1],[1,5,6,1],[1,5,4,1],[1,5,4,1],[1,5,10,1],[1,5,5,1],[1,5,2,1],[1,5,4,1],[1,5,4,1],[1,5,9,1],[1,5,3,1],[1,5,10,1],[1,5,2,1],[1,5,2,1],[1,5,9,1],[1,4,8,1],[1,4,6,1],[1,4,0,1],[1,4,10,1],[1,4,5,1],[1,4,10,1],[1,4,9,1],[1,4,1,1],[1,4,4,1],[1,4,4,1],[1,4,0,1],[1,4,3,1],[1,4,1,1],[1,4,3,1],[1,4,2,1],[1,4,4,1],[1,4,4,1],[1,4,8,1],[1,4,2,1],[1,4,4,1],[1,3,2,1],[1,3,6,1],[1,3,4,1],[1,3,7,1],[1,3,4,1],[1,3,1,1],[1,3,10,1],[1,3,3,1],[1,3,4,1],[1,3,7,1],[1,3,5,1],[1,3,6,1],[1,3,1,1],[1,3,6,1],[1,3,10,1],[1,3,2,1],[1,3,4,1],[1,3,2,1],[1,3,1,1],[1,3,5,1],[1,2,4,1],[1,2,2,1],[1,2,8,1],[1,2,3,1],[1,2,1,1],[1,2,9,1],[1,2,10,1],[1,2,9,1],[1,2,4,1],[1,2,5,1],[1,2,0,1],[1,2,9,1],[1,2,9,1],[1,2,0,1],[1,2,1,1],[1,2,1,1],[1,2,4,1],[1,1,0,1],[1,1,2,1],[1,1,2,1],[1,1,5,1],[1,1,3,1],[1,1,10,1],[1,1,6,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,4,1],[1,1,9,1],[1,1,9,1],[1,1,4,1],[1,1,2,1],[1,1,9,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,1,1],[1,1,1,1],[1,1,5,1]]
daily_minutes_good = [68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,54.01,38.79,47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,35.57,32.98,35,26.07,23.77,39.73,40.57,31.65,31.21,36.32,20.45,21.93,26.02,27.34,23.49,46.94,30.5,33.8,24.23,21.4,27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,30.86,36.07,28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,19.85,35.37,44.6,17.23,13.47,26.33,35.02,32.09,24.81,19.33,28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,39.72,40.8,26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,14.18,35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,18.79,27.61,27.21,26.61,29.77,20.59,27.53,13.82,33.2,25,33.1,36.65,18.63,14.87,22.2,36.81,25.53,24.62,26.25,18.21,28.08,19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,32.51,25.63,22.22,19,32.73,15.16,13.9,27.2,32.01,29.27,33,13.74,20.42,27.32,18.23,35.35,28.48,9.08,24.62,20.12,35.26,19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,33.2,31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,21.67,31.09,33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]

random.seed(0)
beta = estimate_beta(x, daily_minutes_good)
minutes = "minutes = {0}".format(beta[0])
friends = "+ {0} friends".format(beta[1])
work_hours = "{0} work hours".format(beta[2])
phd = "+ {0} PhD".format(beta[3])
print(minutes, friends, work_hours, phd)

minutes = 30.619881701311712 + 0.9702056472470465 friends -1.8671913880379478 work hours + 0.9163711597955347 PhD


So how do we interpret the model?  Let's think of the coefficients of the model as representing all-else-being-equal estimates of the impacts of each factor.  All else being equal, each additinoal friend corresponds to an extra minute spent on the site each day.  All else being equal, each additional hour in a user's workday corresponds to about two fewer minutes spent on the site each day.  All else being equal, having a PhD is associated with spending an extra minute on the site each day.

What this doesn't tell us is anything about the interactions among the variables.  It's possible that the effect of work hours is different for people with many friends than it is for people with few friends.  This model doesn't capture that.  One way to handle this case is to introduce a new variable that is the *product* of "friends" and "work hours".  This effectively allows the "work hours" coefficient to increase or decrease as the number of friends fluctuates.

Additionally, there may be a limit to how much time is spent depending on the coefficients.  We can account for this by adding yet another variable that is the *square* of the number of friends.  Once we start adding new variables, we'll need to begin worrying about whether or not they "matter".  There are no limits to the numbers of products, logs, squares, and higher powers we could add.

If we look at the R-Squared, we'll see it has now increased to 0.68:

In [17]:
def multiple_r_squared(x, y, beta):
    sum_of_squared_errors = sum(error(x_i, y_i, beta) ** 2
                                for x_i, y_i in zip(x, y))
    return 1.0 - sum_of_squared_errors / total_sum_of_squares(y)

multiple_r_squared(x, daily_minutes_good, beta)

0.6800074955952597

Keep in mind that adding new variables to a regression will *necessarily* increase the R-Squared.  The simple regression model is just the special case of the multiple regression model where the coefficients on "work hours" and "PhD" both equal 0.  The optimal multiple regression model will necessarily have an error at least as small as that one.

Because of this, in a multiple regression, we also need to look at the *standard errors of the coefficients*, which measure how certain we are about our estimates of each $ \beta_i $.  The regression as a whole may fit our data very well, but if some of the independent variables are correlated (or irrelevant), their coefficients might not mean much.

The typical approach to measuring these errors starts with another assumption.  The errors $ \epsilon_i $ are independent normal random variables with mean 0 and some shared unknown standard deviation $ \sigma $.  In that case, we can use some linear algebra to find the standard error of each coefficient.  Unfortunately, we're not set up to do that kind of linear algebra from scratch.

So, imagine that we have a sample `n` data points, generated by some distribution:

```python
data = get_sample(num_points=n)
```

We've already written a function to compute the median of the observed data, which we can use as an estimate of the median of the distribution itself.  But how confident can we be about our estimate?  If all the data in the sample are very close to 100, then it seems likely that the actual median is close to 100.  If approximately half the data in the sample is close to 0 and the other half is close to 200, then we can't be nearly as certain about the median.  If we could repeatedly get new samples, we could compute the median of each and look at the distribution of those medians.  Usually we can't.  What we can do instead is *bootstrap* new data sets by choosing `n` data points with replacement from our data and then compute the mediansn of those synthetic data sets:

In [18]:
def bootstrap_sample(data):
    """randomly samples len(data) elements with replacement"""
    return [random.choice(data) for _ in data]

def bootstrap_statistic(data, stats_fn, num_samples):
    """evaluates stats_fn on num_samples bootstrap samples from data"""
    return [stats_fn(bootstrap_sample(data))
            for _ in range(num_samples)]

# 101 points all very close to 100
close_to_100 = [99.5 + random.random() for _ in range(101)]

# 101 points, 50 of them near 0, 50 of them near 200
far_from_100 = ([99.5 + random.random()] +
                [random.random() for _ in range(50)] +
                [200 + random.random() for _ in range(50)])

print("bootstrap_statistic(close_to_100, median, 100):")
print(bootstrap_statistic(close_to_100, median, 100))
print("bootstrap_statistic(far_from_100, median, 100):")
print(bootstrap_statistic(far_from_100, median, 100))

bootstrap_statistic(close_to_100, median, 100):
[100.02634537429101, 100.06586584030461, 100.06850285070729, 100.06586584030461, 100.08022923777746, 100.02634537429101, 100.0251392489555, 100.06586584030461, 100.0251392489555, 100.18315165595742, 100.06627711829937, 100.06586584030461, 100.06627711829937, 100.06627711829937, 100.08022923777746, 100.08022923777746, 99.97928565593025, 99.95763666335068, 100.06627711829937, 99.95763666335068, 100.06586584030461, 100.06627711829937, 100.06850285070729, 100.08732646013036, 100.0340458696825, 100.02634537429101, 100.16675086620182, 100.11133261277095, 100.02634537429101, 100.0340458696825, 100.02048509159759, 100.06586584030461, 100.0340458696825, 100.02048509159759, 100.06850285070729, 100.06850285070729, 99.95763666335068, 100.08022923777746, 100.02048509159759, 100.07674830805912, 100.06586584030461, 100.06850285070729, 100.06627711829937, 100.11133261277095, 100.08022923777746, 100.06627711829937, 100.02634537429101, 100.0251392489555, 1

The `standard_deviation` of the first set of medians is close to 0, while the `standard_deviation` of the second set of medians is close to 100.  We can take the same approach to estimating the standard errors of our regression coefficients.  We repeatedly take a `bootstrap_sample` of our data and estimate `beta` based on that sample.  If the coefficient corresponding to one of the independent variables, say `num_friends`, doesn't vary much across samples, then we can be confident that our estimate is relatively tight.  If the coefficient varies greatly across samples, then we can't be at all confident in our estimate.

The only subtlety is that before sampling, we'll need to `zip` our `x` data and `y` data to make sure that corresponding values of the independent and dependent variables are sampled together.  This means that `bootstrap_sample` will return a list of pairs(`x_i, y_i`), which we'll need to reassemble into an `x_sample` and a `y_sample`.

In [21]:
def estimate_sample_beta(sample):
    x_sample, y_sample = list(zip(*sample)) # magic unzipping trick
    return estimate_beta(x_sample, y_sample)

random.seed(0) # so that you get the same results as me

bootstrap_betas = bootstrap_statistic(list(zip(x, daily_minutes_good)),
                                      estimate_sample_beta,
                                      100)

bootstrap_standard_errors = [
    standard_deviation([beta[i] for beta in bootstrap_betas])
    for i in range(4)]

print(bootstrap_standard_errors)
constant_term = "constant term: {}".format(bootstrap_standard_errors[0])
num_friends = "number of friends: {}".format(bootstrap_standard_errors[1])
unemployed = "unemployed: {}".format(bootstrap_standard_errors[2])
phd = "phd: {}".format(bootstrap_standard_errors[3])
print(constant_term)
print(num_friends)
print(unemployed)
print(phd)

[0.953551702104508, 0.06288763616183773, 0.11722269488203318, 0.8591786495949066]
constant term: 0.953551702104508
number of friends: 0.06288763616183773
unemployed: 0.11722269488203318
phd: 0.8591786495949066


We can use these to test hypotheses such as "does $ \beta_i $ equal zero?"  Under the null hypothesis $ \beta_i = 0 $ and with our other assumptions about the distribution of $ \epsilon_i $ the statistic:
$$ t_j = \beta_j/\sigma_j $$

which is our estimate of $ \beta_j $ divided by our estimate of its standard error, follows a t-distribution with "`n-k` degrees of freedom."  We could create a `students_t_cdf` function to compute p-values for each least-squares coefficient to indicate how likely we would be to observe such a value if the actual coefficient were zero.  But that's outside of scope.  As degrees of freedom get large, the t-distribution gets closer and closer to a standard normal.  In a situation like this, where `n` is much larger than `k`, we can use `normal_cdf` and still feel good about ourselves:

In [22]:
def p_value(beta_hat_j, sigma_hat_j):
    if beta_hat_j > 0:
        return 2 * (1 - normal_cdf(beta_hat_j / sigma_hat_j))
    else:
        return 2 * normal_cdf(beta_hat_j / sigma_hat_j)

print("p_value(30.63, 1.174)", p_value(30.63, 1.174))
print("p_value(0.972, 0.079)", p_value(0.972, 0.079))
print("p_value(-1.868, 0.131)", p_value(-1.868, 0.131))
print("p_value(0.911, 0.990)", p_value(0.911, 0.990))

p_value(30.63, 1.174) 0.0
p_value(0.972, 0.079) 0.0
p_value(-1.868, 0.131) 0.0
p_value(0.911, 0.990) 0.35746719881669264


While most of the coefficients have very small p-values, the coefficient for "PhD" is not "significantly" different from zero, which makes it likely that the coefficient for "PhD" is random rather than meaningful.  In more elaborate regression scenarios, you sometimes want to test more elaborate hypotheses about the data, which you can do with an F-Test, which is also outside scope.

In practice, you'd often like to apply linear regression to data sets with large numbers of variables.  This creates a couple of extra issues.  First, the more variables you use, the more likely you are to overfit your model to the training set.  And second, the more nonzero coefficients you have, the harder it is to make sense of them if the goal is to explain some phenomenon, a sparse model with three factors might be more useful than a slightly better model with hundreds.

*Regularization* is an approach in which we add to the error term a penalty that gets larger as `beta` gets larger.  We then minimize the combined error and penalty.  The more importance we place on the penalty term, the more we discourage large coefficients.  For example:

In [23]:
# alpha is a *hyperparameter* controlling how harsh the penalty is
# sometimes it's called "lambda" but that already means something in Python
def ridge_penalty(beta, alpha):
  return alpha * dot(beta[1:], beta[1:])

def squared_error_ridge(x_i, y_i, beta, alpha):
    """estimate error plus ridge penalty on beta"""
    return error(x_i, y_i, beta) ** 2 + ridge_penalty(beta, alpha)

def ridge_penalty_gradient(beta, alpha):
    """gradient of just the ridge penalty"""
    return [0] + [2 * alpha * beta_j for beta_j in beta[1:]]

def squared_error_ridge_gradient(x_i, y_i, beta, alpha):
    """the gradient corresponding to the ith squared error term
    including the ridge penalty"""
    return vector_add(squared_error_gradient(x_i, y_i, beta),
                      ridge_penalty_gradient(beta, alpha))

def estimate_beta_ridge(x, y, alpha):
    """use gradient descent to fit a ridge regression
    with penalty alpha"""
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(partial(squared_error_ridge, alpha=alpha),
                               partial(squared_error_ridge_gradient,
                                       alpha=alpha),
                               x, y,
                               beta_initial,
                               0.001)

With `alpha` set to zero, there's no penalty at all and we get the same results as before:

In [26]:
random.seed(0)
beta_0 = estimate_beta_ridge(x, daily_minutes_good, alpha=0.0)
print("beta_0 = {0}".format(beta_0))
print("dot = {0}".format(dot(beta_0[1:], beta_0[1:])))
print("multiple_r_squared = {0}".format(multiple_r_squared(x, daily_minutes_good, beta_0)))

beta_0 = [30.619881701311712, 0.9702056472470465, -1.8671913880379478, 0.9163711597955347]
dot = 5.267438780018153
multiple_r_squared = 0.6800074955952597


As we increase `alpha`, the goodness of fit gets worse, but the size of `beta` gets smaller:

In [27]:
for alpha in [0.0, 0.01, 0.1, 1, 10]:
    beta = estimate_beta_ridge(x, daily_minutes_good, alpha=alpha)
    print("alpha", alpha)
    print("beta", beta)
    print("dot(beta[1:],beta[1:])", dot(beta[1:], beta[1:]))
    print("r-squared", multiple_r_squared(x, daily_minutes_good, beta))
    print()

alpha 0.0
beta [30.640219204857612, 0.967957481356752, -1.868740132988584, 0.9147860085860332]
dot(beta[1:],beta[1:]) 5.265964811861464
r-squared 0.6800017437926997

alpha 0.01
beta [30.624475435920044, 0.9671382470586483, -1.8600056698394003, 0.8908465529845938]
dot(beta[1:],beta[1:]) 5.188585061722923
r-squared 0.6799988103788881

alpha 0.1
beta [30.90734173857905, 0.9447829718748573, -1.8488162568510678, 0.5025503230840757]
dot(beta[1:],beta[1:]) 4.56329324277339
r-squared 0.6796752768537219

alpha 1
beta [30.67561351146747, 0.9059985240061532, -1.692308153612175, 0.08524712362724758]
dot(beta[1:],beta[1:]) 3.692007284370296
r-squared 0.6755890872793071

alpha 10
beta [28.341832707881014, 0.7303604899789848, -0.9137372493170081, -0.01795074775901734]
dot(beta[1:],beta[1:]) 1.368664435456863
r-squared 0.5737416601918544



Another approach is lasso regression:

In [28]:
def lasso_penalty(beta, alpha):
    return alpha * sum(abs(beta_i) for beta_i in beta[1:])

Where the ridge penalty shrank the coefficients overall, the lasso penalty tends to force coefficients to be zero, which makes it good for learning sparse models.  Unfortunately, it's not amenable to gradient descent, which means that we won't be able to solve it from scratch.